# Testing 
(Mainly locally, I don't know a method to get python notebook on vader5)
## Checking whether the code interpreter can be called

In [95]:

import requests

base_url = 'http://localhost:8502/api/chatbot'

auth_key = "qA94VhroHMHFN55inWgfAAkt1WEmzQ4J" # Only for testing
auth_string = "&auth_key=" + auth_key


# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to use matplotlib to visualize a sine wave from -2pi to +2pi."
# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following two lines of code: `import numpy as py` `import matplotlib.pyplot as plt` " # LLM likes to break this into two code_interpreter calls, which I don't currently handle correctly.
# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following code: \"import numpy as np\nimport matplotlib.pyplot as plt\n\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\". Do not add anything else, plt.show() for example will not work." # doesn't work
# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following code: \"import numpy as np\nimport matplotlib.pyplot as plt\n\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10,5))\nplt\". Do not add anything else, plt.show() for example will not work." # doesn't work
user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following code: \"import numpy as np\nimport matplotlib.pyplot as plt\nimport matplotlib\nmatplotlib.use('agg')\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()\"."
url = base_url + '/streamresponse?input=' + user_input + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
complete_response = [] # The stream gets consumed when streamed, we'll store it here.

# Note that this way to cunsume the response chunks it and those chunks are surprisingly small, cutting up the response.
# A better way would be `for delta in response.iter_content(chunk_size=1024):`
for delta in response:
    print(delta)
    complete_response.append(delta.decode("utf-8"))

b'{"variant":"ServerHint","content":"thread_id:Ki3jezoa4kjoWdYSEaEVkAgpaxQ8kvaD"}'
b'{"variant":"Code","content":["","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":["{\\"","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":["code","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":["\\":\\"","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":["import","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":[" numpy","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":[" as","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":[" np","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":["\\\\n","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":["import","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":[" matplotlib","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","content":[".pyplot","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}'
b'{"variant":"Code","conten

In [96]:
# Some elements are cut off early if the chunk size is not specified or too small, so to reconstruct, we'll need to append all element that don't start with "{" to the previous element.
reconstructed_response = []
for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]

for delta in reconstructed_response:
    print(delta)

{"variant":"ServerHint","content":"thread_id:Ki3jezoa4kjoWdYSEaEVkAgpaxQ8kvaD"}
{"variant":"Code","content":["","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":["{\"","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":["code","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":["\":\"","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":["import","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":[" numpy","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":[" as","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":[" np","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":["\\n","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":["import","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":[" matplotlib","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":[".pyplot","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"variant":"Code","content":[" as","call_kLUDXXebVUNuBBJ8FUXZPV7D"]}
{"

In [97]:
# Now we can parse the response as JSON
import json
response_json = [json.loads(x) for x in reconstructed_response]
for element in response_json:
    print(element)

{'variant': 'ServerHint', 'content': 'thread_id:Ki3jezoa4kjoWdYSEaEVkAgpaxQ8kvaD'}
{'variant': 'Code', 'content': ['', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': ['{"', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': ['code', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': ['":"', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': ['import', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': [' numpy', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': [' as', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': [' np', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': ['\\n', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': ['import', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': [' matplotlib', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'content': ['.pyplot', 'call_kLUDXXebVUNuBBJ8FUXZPV7D']}
{'variant': 'Code', 'co

In [98]:
# We can use this to easily extract only the Assistant's responses
assistant_responses = [x for x in response_json if x["variant"] == "Assistant"]
print("".join([x["content"] for x in assistant_responses])) # This will print the Assistant's responses in order.

It seems that the code execution did not produce any output. Let's try displaying the plot without using `matplotlib.use('agg')`, which is typically used for non-interactive backends. I will modify the code slightly and run it again. 

Let's proceed with this adjustment.The code execution has been successful! The plot of the sine wave from -2π to 2π has been generated. The plot features the sine wave properly labeled with the title and axes, including a grid and reference lines at y=0 and x=0. 

If you would like to conduct further tests or explore any specific functionalities, please let me know!


In [99]:
# Or the generated Code
code_responses = [x for x in response_json if x["variant"] == "Code"]
print("".join([x["content"][0] for x in code_responses])) # This will print the generated code in order.

{"code":"import numpy as np\nimport matplotlib.pyplot as plt\nimport matplotlib\nmatplotlib.use('agg')\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()"}{"code":"import numpy as np\nimport matplotlib.pyplot as plt\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()"}


In [100]:
# Or all coded output
code_output_responses = [x["content"][0] for x in response_json if x["variant"] == "CodeOutput"]
for error in code_output_responses:
    print(error)

In [101]:
# Testing the get_thread endpoint
thread_id = "dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw" # for testing
url = base_url + '/getthread?thread_id=' + thread_id + auth_string
response = requests.get(url)
print(response.json())

[{'variant': 'ServerHint', 'content': 'thread_id:dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw'}, {'variant': 'User', 'content': 'This is a test regarding your capabilities of using the code_interpreter tool. Please call the code_interpreter tool with the input \\"2+2\\".'}, {'variant': 'Code', 'content': ['\\"', '\\']}, {'variant': 'CodeOutput', 'content': ['The code interpreter was successfully called, but is not yet implemented. The inputs to this function were: code_interpreter ; Some(\\"{\\\\\\"code\\\\\\"', '\\\\\\"2+2\\\\\\"}\\"). @FrevaGPT, you may tell the user that the tool call was called and interpreted, but not yet implemented.']}, {'variant': 'OpenAIError', 'content': 'Error getting response.'}, {'variant': 'StreamEnd', 'content': 'Stream ended abruptly'}]
